In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

Using TensorFlow backend.


In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

In [4]:
dataset = df.values

In [5]:
X = []
for ind in df.index:
    X.append(df.loc[ind].CT)

X = np.array(X)/10
X = X.reshape(X.shape[0], 512, 512, 1)

In [6]:
from sklearn.preprocessing import MinMaxScaler
y = result.values
scaler = MinMaxScaler()
scaler.fit(y)
y_scaled = scaler.transform(y)

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2)

In [48]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(64, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu', 
                 input_shape=(512,512,1), kernel_initializer='random_normal', 
                 bias_initializer='zeros' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='relu'))
# output layer
model.add(Dense(1, activation='sigmoid'))

In [49]:
model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam')

In [ ]:
model.fit(X_train,y_train, batch_size=32, epochs=20, validation_data=(X_test,y_test))

Train on 140 samples, validate on 36 samples
Epoch 1/20
140/140 [==============================] - 9s 64ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0219 - val_mean_squared_error: 0.0219
Epoch 2/20
140/140 [==============================] - 8s 58ms/step - loss: 0.0198 - mean_squared_error: 0.0198 - val_loss: 0.0222 - val_mean_squared_error: 0.0222
Epoch 3/20
140/140 [==============================] - 8s 59ms/step - loss: 0.0198 - mean_squared_error: 0.0198 - val_loss: 0.0224 - val_mean_squared_error: 0.0224
Epoch 4/20
140/140 [==============================] - 8s 59ms/step - loss: 0.0187 - mean_squared_error: 0.0187 - val_loss: 0.0222 - val_mean_squared_error: 0.0222
Epoch 5/20
140/140 [==============================] - 8s 59ms/step - loss: 0.0181 - mean_squared_error: 0.0181 - val_loss: 0.0222 - val_mean_squared_error: 0.0222
Epoch 6/20
140/140 [==============================] - 8s 59ms/step - loss: 0.0163 - mean_squared_error: 0.0163 - val_loss: 0.0222 - val_mean

In [41]:
scaler.inverse_transform(model.predict(X_test))

array([[-7.4007325 ],
       [-6.494286  ],
       [-6.3619285 ],
       [-8.007622  ],
       [-6.6625533 ],
       [-7.899496  ],
       [-2.8914237 ],
       [-1.2078733 ],
       [-0.78241044],
       [-3.1333277 ],
       [-4.6602025 ],
       [-9.227251  ],
       [-3.884321  ],
       [-8.494724  ],
       [-6.87902   ],
       [-3.6849215 ],
       [-6.2760487 ],
       [-3.9387085 ],
       [-5.679025  ],
       [-7.2061234 ],
       [-3.1693962 ],
       [-8.906324  ],
       [-7.75552   ],
       [-6.0269012 ],
       [-8.220977  ],
       [-6.70435   ],
       [-5.4434752 ],
       [-0.9539635 ],
       [-6.1597133 ],
       [-8.514712  ],
       [-1.8876113 ],
       [-1.6841084 ],
       [-4.637399  ],
       [-1.4360673 ],
       [-6.511016  ],
       [-2.393571  ]], dtype=float32)

In [42]:
n=8
example = df.iloc[n].CT.reshape(1,512,512,1)
print(scaler.inverse_transform(model.predict(example)))
print(result.iloc[n])

[[12.922438]]
slope   -3.41031
Name: ID00229637202260254240583, dtype: object


In [46]:
abs(scaler.inverse_transform(model.predict(X_test)) - scaler.inverse_transform(y_test)).mean()

3.409579953933438

In [29]:
scaler.inverse_transform(y_test)

array([[  0.35006384],
       [ -2.13597507],
       [-12.68318624],
       [ -0.06711805],
       [  0.32038146],
       [ -0.95680346],
       [ -0.33056823],
       [ -5.94791091],
       [ -5.9863249 ],
       [  7.19713765],
       [  4.67761641],
       [ -2.35611511],
       [  7.33125819],
       [ -0.26516073],
       [ -0.28084152],
       [ -2.54915177],
       [ -6.74182279],
       [-11.62740016],
       [-18.38177853],
       [ -1.0813172 ],
       [  0.69377163],
       [ -3.81404806],
       [ -5.73331844],
       [-23.2875    ],
       [ -0.57674104],
       [  8.24164244],
       [ -9.38184773],
       [ -6.04144002],
       [  1.89659994],
       [ -4.80809129],
       [ -2.58405342],
       [ -8.93189583],
       [ -4.28962749],
       [ -1.56416425],
       [ -1.8502907 ],
       [-10.81815282],
       [  0.71656977],
       [ -4.40627448],
       [-14.03341163],
       [ -6.40337866],
       [ -0.19162534],
       [ -3.0797419 ],
       [ -4.24042951],
       [ -7